# Playing with Elasticsearch via REST

In those examples you will see how clients interacts with Elasticsearch unsing REST APIs. 

An index is defined using the Create Index API, which can be accomplished with a simple **PUT** command:

In [3]:
import requests
import json
import pprint

req = requests.put('http://elasticsearch:9200/my_test')
print(req.json())

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_test'}


To store a document on elasticsearch you should use **PUT** again, notice that in this example you are specifying a **unique id** of the document.

In [4]:
document = {'username':'Alice',
            'comment':'I love to see the stars at night'}

requestResponse = requests.put('http://elasticsearch:9200/my_test/doc/1',json=document)
pprint.pprint(requestResponse.json())


{'_id': '1',
 '_index': 'my_test',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'doc',
 '_version': 1,
 'result': 'created'}


You can avoid specifying the document id but using **POST** method. In this case elasticsearch create one

In [5]:
document = {'username':'Maria',
            'comment':'My favorite painting is Starry Night'}

requestResponse = requests.post('http://elasticsearch:9200/my_test/doc/',json=document)
pprint.pprint(requestResponse.json())

{'_id': 'g0T8dnYBAVlRoj4jeWgI',
 '_index': 'my_test',
 '_primary_term': 1,
 '_seq_no': 1,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'doc',
 '_version': 1,
 'result': 'created'}


To obtain a document you need to use **GET** method. 

In [6]:
storedDocument = requests.get('http://elasticsearch:9200/my_test/doc/1',json=document)
pprint.pprint(storedDocument.json())


{'_id': '1',
 '_index': 'my_test',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'comment': 'I love to see the stars at night',
             'username': 'Alice'},
 '_type': 'doc',
 '_version': 1,
 'found': True}


But you can also launch queries over elasticsearch, using **GET** method and with a specific json that contain **query** and **match** clauses. Here you can see how you can obtain all documents from an index. 

In [7]:
document = { 'query': {
                'match_all': {}
              }
           }

storedDocuments = requests.get('http://elasticsearch:9200/my_test/_search',json=document)
pprint.pprint(storedDocuments.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'my_test',
                    '_score': 1.0,
                    '_source': {'comment': 'I love to see the stars at night',
                                'username': 'Alice'},
                    '_type': 'doc'},
                   {'_id': 'g0T8dnYBAVlRoj4jeWgI',
                    '_index': 'my_test',
                    '_score': 1.0,
                    '_source': {'comment': 'My favorite painting is Starry '
                                           'Night',
                                'username': 'Maria'},
                    '_type': 'doc'}],
          'max_score': 1.0,
          'total': {'relation': 'eq', 'value': 2}},
 'timed_out': False,
 'took': 7}


To store a document you can use **Create** or **Index** methods. Create will try to store a document if this id is not already present. 

In [8]:
document = {'username':'Alice',
            'comment':'I love to see the stars and the moon at night'}

requestResponse = requests.put('http://elasticsearch:9200/my_test/doc/1/_create',json=document)
pprint.pprint(requestResponse.json())

{'error': {'index': 'my_test',
           'index_uuid': 'lsZrJLmvRv2nx0KAiDmrsw',
           'reason': '[1]: version conflict, document already exists (current '
                     'version [1])',
           'root_cause': [{'index': 'my_test',
                           'index_uuid': 'lsZrJLmvRv2nx0KAiDmrsw',
                           'reason': '[1]: version conflict, document already '
                                     'exists (current version [1])',
                           'shard': '0',
                           'type': 'version_conflict_engine_exception'}],
           'shard': '0',
           'type': 'version_conflict_engine_exception'},
 'status': 409}


However, if you use index it directly, the document will be overriten (without _create sufix). 

In [9]:
document = {'username':'Alice',
            'comment':'I love to see the stars and the moon at night'}

requestResponse = requests.put('http://elasticsearch:9200/my_test/doc/1',json=document)
pprint.pprint(requestResponse.json())

{'_id': '1',
 '_index': 'my_test',
 '_primary_term': 1,
 '_seq_no': 2,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'doc',
 '_version': 2,
 'result': 'updated'}


In [10]:
storedDocument = requests.get('http://elasticsearch:9200/my_test/doc/1',json=document)
pprint.pprint(storedDocument.json())

{'_id': '1',
 '_index': 'my_test',
 '_primary_term': 1,
 '_seq_no': 2,
 '_source': {'comment': 'I love to see the stars and the moon at night',
             'username': 'Alice'},
 '_type': 'doc',
 '_version': 2,
 'found': True}


## Ejecicio 1

Create an index called cars

In [11]:
req = requests.put('http://elasticsearch:9200/cars')
print(req.json())

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'cars'}


Insert a car document with following schema:
    {
        id: 0000LZD
        model: Tesla
        year: 2020
        color: Blue
    }

In [17]:
document = {'id':'0000LZD',
            'model':'Tesla',
            'year':'2020',
            'color':'Blue'}

requestResponse = requests.put('http://elasticsearch:9200/cars/doc/0000LZD',json=document)
pprint.pprint(requestResponse.json())

{'_id': '0000LZD',
 '_index': 'cars',
 '_primary_term': 1,
 '_seq_no': 1,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'doc',
 '_version': 2,
 'result': 'updated'}


Run a query to get the car and print on the screen

In [21]:
storedDocuments = requests.get('http://elasticsearch:9200/cars/doc/0000LZD',)
pprint.pprint(storedDocuments.json())

{'_id': '0000LZD',
 '_index': 'cars',
 '_primary_term': 1,
 '_seq_no': 1,
 '_source': {'color': 'Blue',
             'id': '0000LZD',
             'model': 'Tesla',
             'year': '2020'},
 '_type': 'doc',
 '_version': 2,
 'found': True}


In [22]:
print(storedDocuments.json())

{'_index': 'cars', '_type': 'doc', '_id': '0000LZD', '_version': 2, '_seq_no': 1, '_primary_term': 1, 'found': True, '_source': {'id': '0000LZD', 'model': 'Tesla', 'year': '2020', 'color': 'Blue'}}


## Ejercicio 2

https://docs.google.com/document/d/1sLMzF1RcI1Dss3oZEGYD7e50j0ZSoIr-oF_sU4tFvG0

### Apartado G

In [24]:
req = requests.get('http://elasticsearch:9200/products/_search',)
pprint.pprint(req.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '53879',
                    '_index': 'products',
                    '_score': 1.0,
                    '_source': {'brandName': 'Reach',
                                'customerRating': 2,
                                'grp_id': '53879',
                                'price': 10.84,
                                'productName': 'Reach Floss Waxed Mint',
                                'quantitySold': 652351,
                                'type': 'csv',
                                'upc12': '381370092292'},
                    '_type': 'product'},
                   {'_id': '53880',
                    '_index': 'products',
                    '_score': 1.0,
                    '_source': {'brandName': 'Our Family',
                                'customerRating': 3,
                                'grp_id': '53880',
                                'price': 2.83,
             

### Apartado h

In [25]:
req = requests.get('http://elasticsearch:9200/products/_search?pretty',)
pprint.pprint(req.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '53879',
                    '_index': 'products',
                    '_score': 1.0,
                    '_source': {'brandName': 'Reach',
                                'customerRating': 2,
                                'grp_id': '53879',
                                'price': 10.84,
                                'productName': 'Reach Floss Waxed Mint',
                                'quantitySold': 652351,
                                'type': 'csv',
                                'upc12': '381370092292'},
                    '_type': 'product'},
                   {'_id': '53880',
                    '_index': 'products',
                    '_score': 1.0,
                    '_source': {'brandName': 'Our Family',
                                'customerRating': 3,
                                'grp_id': '53880',
                                'price': 2.83,
             

### Apartado n

In [26]:
docu = {
"size": 25,
  "query": {
    "match": {
      "productName": "peanut butter"
    }
  }
}
req = requests.get('http://elasticsearch:9200/products/_search', json=docu)
pprint.pprint(req.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '108162',
                    '_index': 'products',
                    '_score': 11.242394,
                    '_source': {'brandName': 'Arrowhead Mills',
                                'customerRating': 3,
                                'grp_id': '108162',
                                'price': 19.36,
                                'productName': 'Peanut Butter Creamy',
                                'quantitySold': 655728,
                                'type': 'csv',
                                'upc12': '074333470441'},
                    '_type': 'product'},
                   {'_id': '95241',
                    '_index': 'products',
                    '_score': 11.242394,
                    '_source': {'brandName': 'Unreal',
                                'customerRating': 3,
                                'grp_id': '95241',
                                'price': 5

### Apartado p

In [27]:
docu = {
"size": 25,
  "query": {
    "match": {
      "productName": "Peanut Butter"
    }
  }
}
req = requests.get('http://elasticsearch:9200/products/_search', json=docu)
pprint.pprint(req.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '108162',
                    '_index': 'products',
                    '_score': 11.242394,
                    '_source': {'brandName': 'Arrowhead Mills',
                                'customerRating': 3,
                                'grp_id': '108162',
                                'price': 19.36,
                                'productName': 'Peanut Butter Creamy',
                                'quantitySold': 655728,
                                'type': 'csv',
                                'upc12': '074333470441'},
                    '_type': 'product'},
                   {'_id': '95241',
                    '_index': 'products',
                    '_score': 11.242394,
                    '_source': {'brandName': 'Unreal',
                                'customerRating': 3,
                                'grp_id': '95241',
                                'price': 5

### Apartado q

In [28]:
docu = {
"size": 25,
  "query": {
    "match": {
      "productName": "kasmati rice"
    }
  }
}
req = requests.get('http://elasticsearch:9200/products/_search', json=docu)
pprint.pprint(req.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '27738',
                    '_index': 'products',
                    '_score': 19.586052,
                    '_source': {'brandName': 'Riceselect',
                                'customerRating': 2,
                                'grp_id': '27738',
                                'price': 13.68,
                                'productName': 'Rice Select Kasmati Rice',
                                'quantitySold': 267886,
                                'type': 'csv',
                                'upc12': '074401810414'},
                    '_type': 'product'},
                   {'_id': '89902',
                    '_index': 'products',
                    '_score': 6.45659,
                    '_source': {'brandName': 'Louisiana Purchase',
                                'customerRating': 1,
                                'grp_id': '89902',
                                'pr

### Apartado r

In [29]:
docu = {
  "query": {
    "match": {
      "customerRating": 4
    }
  }
}
req = requests.get('http://elasticsearch:9200/products/_search', json=docu)
pprint.pprint(req.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '53881',
                    '_index': 'products',
                    '_score': 1.0,
                    '_source': {'brandName': 'Guaranteed Value',
                                'customerRating': 4,
                                'grp_id': '53881',
                                'price': 14.79,
                                'productName': 'Guaranteed Value Ice Cream '
                                               'Chocolate Marshmallow Swirl',
                                'quantitySold': 182849,
                                'type': 'csv',
                                'upc12': '688267065408'},
                    '_type': 'product'},
                   {'_id': '53882',
                    '_index': 'products',
                    '_score': 1.0,
                    '_source': {'brandName': 'Burpee',
                                'customerRating': 4,
                      

### Apartado s

In [31]:
docu = {
  "query": {
    "match": {
      "price": 10
    }
  }
}
req = requests.get('http://elasticsearch:9200/products/_search', json=docu)
pprint.pprint(req.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '58886',
                    '_index': 'products',
                    '_score': 1.0,
                    '_source': {'brandName': 'Stonyfield Organic',
                                'customerRating': 3,
                                'grp_id': '58886',
                                'price': 10.0,
                                'productName': 'Stonyfield Organic Butter '
                                               'Unsalted',
                                'quantitySold': 509726,
                                'type': 'csv',
                                'upc12': '052159701000'},
                    '_type': 'product'},
                   {'_id': '59493',
                    '_index': 'products',
                    '_score': 1.0,
                    '_source': {'brandName': 'Betts',
                                'customerRating': 4,
                                'grp_id': 

### Apartado t

In [32]:
docu = {
  "query": {
    "range": {
      "price": {
          "gte": 10
      }
    }
  }
}
req = requests.get('http://elasticsearch:9200/products/_search', json=docu)
pprint.pprint(req.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '53879',
                    '_index': 'products',
                    '_score': 1.0,
                    '_source': {'brandName': 'Reach',
                                'customerRating': 2,
                                'grp_id': '53879',
                                'price': 10.84,
                                'productName': 'Reach Floss Waxed Mint',
                                'quantitySold': 652351,
                                'type': 'csv',
                                'upc12': '381370092292'},
                    '_type': 'product'},
                   {'_id': '53881',
                    '_index': 'products',
                    '_score': 1.0,
                    '_source': {'brandName': 'Guaranteed Value',
                                'customerRating': 4,
                                'grp_id': '53881',
                                'price': 14.79,
      